In [1]:
import mne
import pandas as pd
import numpy as np
def get_ica(signal, mode='sources'):
    """
    mode: 
        sources - projected components x signal (64x265)
        components - ICA components (64x64)
    """;
    
    # define channels
    channel_list = ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd']
    
    # create mne raw object
    info = mne.create_info(channel_list, 256, ch_types='eeg', verbose=False)
    raw = mne.io.RawArray(data=signal, info=info, verbose=False)

    # filter signal
#     raw.filter(l_freq=1., h_freq=None, verbose=False);

    # run ica
    ica = mne.preprocessing.ICA(n_components=64, verbose=False)
    ica.fit(raw, picks=channel_list);
    
    if mode == 'sources':
        return ica.get_sources(raw).get_data()
    elif mode == 'components':
        return norm_min_max(ica.get_components())
    else:
        raise NotImplementedError

In [2]:
train = pd.read_csv("/sise/liorrk-group/OrDanOfir/eeg/data/img_train.csv")

In [3]:
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
def save(row):
    img = np.load(row)['img']
    ica = get_ica(img[:,:,0].T).T
    image = img.copy()
    image[:,:,0] = ica
    np.savez_compressed(row.replace("/images/","/images_ica/"),img=img,ica=image)

In [1]:
%%time
_ = Parallel(n_jobs=16)(delayed(save)(row) for row in tqdm(train.path))

In [5]:

# for row in tqdm(train.path):
#     img = np.load(row)['img']
#     ica = get_ica(img[:,:,0].T).T
#     image = img.copy()
#     image[:,:,0] = ica
#     np.savez_compressed(row.replace("/images/","/images_ica/"),img=img,ica=image)
    